In [1]:
%%capture
!pip install datasets --no-build-isolation
!pip install seqeval
!pip install transformers[torch]

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
import torch
import numpy as np
from datasets import load_metric

# CoNLL-2003 Dataset Description

The CoNLL-2003 dataset is widely used for training and evaluating Named Entity Recognition (NER) models. The dataset focuses on four types of named entities: persons (PER), locations (LOC), organizations (ORG), and miscellaneous entities (MISC).

## Dataset Structure:
Each data file contains four columns separated by a single space:
1. Word
2. Part-of-Speech (POS) tag
3. Syntactic chunk tag
4. Named entity tag

Words are listed on separate lines, and sentences are separated by a blank line.
The chunk and named entity tags follow the IOB2 tagging scheme:
- `B-TYPE`: Beginning of a phrase of type TYPE
- `I-TYPE`: Inside a phrase of type TYPE
- `O`: Outside any named entity phrase

## Example:
```python
{
    "chunk_tags": [11, 12, 12, 21, 13, 11, 11, 21, 13, 11, 12, 13, 11, 21, 22, 11, 12, 17, 11, 21, 17, 11, 12, 12, 21, 22, 22, 13, 11, 0],
    "id": "0",
    "ner_tags": [0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "pos_tags": [12, 22, 22, 38, 15, 22, 28, 38, 15, 16, 21, 35, 24, 35, 37, 16, 21, 15, 24, 41, 15, 16, 21, 21, 20, 37, 40, 35, 21, 7],
    "tokens": ["The", "European", "Commission", "said", "on", "Thursday", "it", "disagreed", "with", "German", "advice", "to", "consumers", "to", "shun", "British", "lamb", "until", "scientists", "determine", "whether", "mad", "cow", "disease", "can", "be", "transmitted", "to", "sheep", "."]
}


## Named Entity Tags
- **O**: Outside a named entity
- **B-PER**: Beginning of a person's name
- **I-PER**: Inside a person's name
- **B-ORG**: Beginning of an organization name
- **I-ORG**: Inside an organization name
- **B-LOC**: Beginning of a location name
- **I-LOC**: Inside a location name
- **B-MISC**: Beginning of miscellaneous entity
- **I-MISC**: Inside a miscellaneous entity


In [3]:
# Load dataset
dataset = load_dataset("conll2003")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [4]:
!mv /root/.cache/huggingface/datasets/conll2003 /content/

In [ ]:
!ls /content/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72/

dataset_info.json      multi_news-train-00000-of-00002.arrow
LICENSE		       multi_news-train-00001-of-00002.arrow
multi_news-test.arrow  multi_news-validation.arrow


In [5]:
from datasets import Dataset

train_dataset = Dataset.from_file("/content/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/conll2003-train.arrow")
test_dataset = Dataset.from_file("/content/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/conll2003-test.arrow")
val_dataset = Dataset.from_file("/content/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/conll2003-validation.arrow")


In [7]:
# Model checkpoint
checkpoint = "bert-base-cased"
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

## Token Labeling in NER: Use of `-100`

In Named Entity Recognition (NER) tasks, the label `-100` is commonly used to signify that certain tokens should be ignored during the loss calculation in model training. This approach helps focus the learning on meaningful parts of the data. Here's an overview of the types of tokens typically assigned a `-100` label:

### 1. **Subsequent Sub-tokens**
After a word is split into multiple sub-tokens, only the first sub-token receives the actual entity label. Subsequent sub-tokens receive `-100` to ensure that entity labels are not incorrectly assigned to fragments of words.

### 2. **Special Tokens**
Special tokens such as `[CLS]`, `[SEP]`, and `[PAD]` used for managing sequence boundaries and lengths in models like BERT are also assigned `-100` as they do not correspond to real words in the text.

### 3. **Non-Entity Tokens**
In certain training setups, tokens that do not correspond to any entity and are not the focus of the task might also be marked with `-100`, especially in cases of imbalanced datasets.

### Example
- **Sentence**: "John lives in New York"
- **Tokens**: ["[CLS]", "John", "lives", "in", "New", "York", "[SEP]"]
- **Labels**: [-100, "B-PER", "O", "O", "B-LOC", "I-LOC", -100]

This labeling strategy is critical for efficient model training, ensuring that the model focuses only on relevant tokens.


In [11]:
# Tokenize and align labels without fixed padding
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_train_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

In [12]:
# Load pre-trained model
model = AutoModelForTokenClassification.from_pretrained(checkpoint, num_labels=9)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


`seqeval` is a Python framework for sequence labeling evaluation. seqeval can evaluate the performance of chunking tasks such as named-entity recognition, part-of-speech tagging, semantic role labeling and so on.

In [13]:
# Load seqeval metric
metric = load_metric("seqeval")

# Define compute_metrics function
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

<ipython-input-13-2746363108>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [15]:
# Get label list
label_list = train_dataset.features["ner_tags"].feature.names

# Set up data collator for dynamic padding
data_collator = DataCollatorForTokenClassification(tokenizer)

In [16]:
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [17]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [19]:
# Initialize Trainer with compute_metrics and data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-19-1788778007>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
# Train model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: r8899814 (r8899814-no) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.108733,0.836125,0.856951,0.846410,0.973560
2,No log,0.096276,0.867238,0.886065,0.876550,0.978739
3,0.156300,0.098333,0.871993,0.890778,0.881285,0.979674


TrainOutput(global_step=648, training_loss=0.128142993391296, metrics={'train_runtime': 196.6841, 'train_samples_per_second': 52.668, 'train_steps_per_second': 3.295, 'total_flos': 263861608299318.0, 'train_loss': 0.128142993391296, 'epoch': 3.0})

In [21]:
# Evaluate model
results = trainer.evaluate()
print(results)

{'eval_loss': 0.09833312779664993, 'eval_precision': 0.871993410214168, 'eval_recall': 0.8907775159878829, 'eval_f1': 0.8812853812853814, 'eval_accuracy': 0.9796736887192866, 'eval_runtime': 13.3204, 'eval_samples_per_second': 243.987, 'eval_steps_per_second': 15.315, 'epoch': 3.0}


## Testing

In [29]:
# Function to run NER on a given sentence
def predict_ner_from_text(text, model, tokenizer, label_list):
    # Tokenize input text
    tokens = tokenizer(text, return_tensors="pt", truncation=True, is_split_into_words=False)
    tokens.to('cuda')
    with torch.no_grad():
        output = model(**tokens)

    predictions = torch.argmax(output.logits, dim=2)[0].tolist()
    input_tokens = tokenizer.convert_ids_to_tokens(tokens["input_ids"][0])

    # Align and print results (skip special tokens)
    for token, pred in zip(input_tokens, predictions):
        if token not in tokenizer.all_special_tokens:
            print(f"{token:15} -> {label_list[pred]}")


# Example usage
test_sentence = "Rahul has fine tuned this BERT model by the lecture from CampusX"
predict_ner_from_text(test_sentence, model, tokenizer, label_list)


Ra              -> B-PER
##hul           -> B-PER
has             -> O
fine            -> O
tuned           -> O
this            -> O
B               -> B-MISC
##ER            -> O
##T             -> O
model           -> O
by              -> O
the             -> O
lecture         -> O
from            -> O
Campus          -> B-ORG
##X             -> I-ORG
